All the basic imports.

In [1]:
import pandas as pd                                         #https://pandas.pydata.org/docs/index.html
from pandas import read_csv,read_json

import imageio
import os

Some constants.

In [2]:
path_input='data/img/'
path_output='data/img_processed/'
image_num='01243'

First I will try to work on text recognition in an image.

Pytesseract is an optical character recognition tool for python. It is a wrapper for tor Google's Tesseract-OCR Engine.

In [3]:
import pytesseract                                          #https://pypi.org/project/pytesseract/

print(pytesseract.image_to_string(path_input+image_num+'.png',lang='eng'))

" * te
When your human’ says: “%
_ “who's agood girlas — el

im o->
4 a7

} >
—

and you already know it's you



Shit doesn't work well out of the box.

It seems I will need to check out OpenCV's threshold blah blah.

I'm sleepy will try tomorrow.

Fuck tomorrow, today it is.

In [4]:
import cv2 as cv                                    #https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html

from cv2 import threshold,adaptiveThreshold         #threshold needs a grayscale image to work properly      
#thresholding-> https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html
#threshold function-> https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57
#adaptiveThreshold function-> https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#ga72b913f352e4a1b1b397736707afcde3

from cv2 import cvtColor                            #changes colorspace based on the flag provided             
#colorspace-> https://docs.opencv.org/4.x/df/d9d/tutorial_py_colorspaces.html
#cvtColor function-> https://docs.opencv.org/4.x/d8/d01/group__imgproc__color__conversions.html#ga397ae87e1288a81d2363b61574eb8cab

from cv2 import COLOR_BGR2GRAY                      #imports the flag for BGR to Grayscale conversion
#list of conversion flags-> 

from cv2 import ADAPTIVE_THRESH_MEAN_C,ADAPTIVE_THRESH_GAUSSIAN_C       #imports the flags for adaptive thresholding
#list of adaptive threshold function flags-> https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#gaa42a3e6ef26247da787bf34030ed772c

from cv2 import THRESH_BINARY,THRESH_BINARY_INV     #imports the flags for types of thresholding
#list of thresholding types flags-> https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#gaa9e58d2860d4afa658ef70a9b1115576

Applying a simple grayscale (B/W) conversion to the image with opencv.

In [13]:
img=cv.imread(path_input+image_num+'.png')
gray=cvtColor(img,COLOR_BGR2GRAY)

if os.path.isdir(path_output):
    cv.imwrite(path_output+image_num+'.png',gray)
else:
    os.mkdir(path_output)
    cv.imwrite(path_output+image_num+'.png',gray)

After the grayscale conversion, tesseract character recognition accuracy has improved (?)

In [6]:
print(pytesseract.image_to_string(path_output+image_num+'.png',lang='eng'))

when your humansays:
_ ‘“who>sa good gira.
Lh « 79

and. you already Know it's you



Time for threshold. After a lot of try and error and many voodoo dolls burned and demons summoned, it works (?)

In [7]:
gray_thres=adaptiveThreshold(gray,255,ADAPTIVE_THRESH_GAUSSIAN_C,THRESH_BINARY,3,9)
cv.imwrite(path_output+image_num+'_t.png',gray_thres)

True

In [8]:
print(pytesseract.image_to_string(path_output+image_num+'_t.png',lang='eng'))

Hie Our IMA Says

“who's a good oi”

and sou alreaaly now i's you




Copied code from G4G gives similar output.

In [9]:
ret, thresh1 = cv.threshold(gray, 0, 255, cv.THRESH_OTSU | cv.THRESH_BINARY_INV)

# Specify structure shape and kernel size. 
# Kernel size increases or decreases the area 
# of the rectangle to be detected.
# A smaller value like (10, 10) will detect 
# each word instead of a sentence.
rect_kernel = cv.getStructuringElement(cv.MORPH_RECT, (18, 18))

# Applying dilation on the threshold image
dilation = cv.dilate(thresh1, rect_kernel, iterations = 1)
contours, hierarchy = cv.findContours(dilation, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

# Creating a copy of image
im2 = gray_thres.copy()

# A text file is created and flushed
file = open("recognized.txt", "w+")
file.write("")
file.close()

# Looping through the identified contours
# Then rectangular part is cropped and passed on
# to pytesseract for extracting text from it
# Extracted text is then written into the text file
for cnt in contours:
    x, y, w, h = cv.boundingRect(cnt)
    
    # Drawing a rectangle on copied image
    rect = cv.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Cropping the text block for giving input to OCR
    cropped = im2[y:y + h, x:x + w]
    
    # Open the file in append mode
    file = open("recognized.txt", "a")
    
    # Apply OCR on the cropped image
    text = pytesseract.image_to_string(cropped)
    
    # Appending the text into file
    file.write(text)
    file.write("\n")
    
    # Close the file
    file.close()


OKAY FUCK TESSERACT, ITS SHIT, GONNA TRY KERAS OCR NEXT

There may be extra imports, will clean it up on final pass

In [34]:
import tensorflow as tf
tf.get_logger().setLevel(0)                         #could be destructive later on, but hey no more screen filler
import keras_ocr                                    #https://keras-ocr.readthedocs.io/en/latest/
import matplotlib.pyplot as plt                     #useful for plotting bounding boxes on image to show OCR
import os

Creating the pre-trained pipeline in keras-ocr

You could train a fresh new one by feeding it a lot of data. Possible, but time-consuming, why do that unless you get paid for it?

In [35]:
pipe=keras_ocr.pipeline.Pipeline()

Looking for C:\Users\Mobius\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\Mobius\.keras-ocr\crnn_kurapan.h5


Creating a list of all the local files that are to be put through the OCR

In [17]:
img_path='data/img/'
indx=os.listdir(img_path)
indx=[img_path+indx[i] for i in range(0,len(indx))]

Reminder: DO NOT put the whole index in the OCR, chews through RAM, need more RAM, 8gigs in not it. Grab a slice. Even 10 images are too much it may seem. I really need more RAM.

It will burn through CPU, so any background processes may/will crash.

In [22]:
test_batch=2
images=[    keras_ocr.tools.read(url_or_path) for url_or_path in indx[:test_batch]  ]
pred_list=pipe.recognize(images)         #initial output by the pipeline is in for format of list[image_list][string_list][tuples], you have to dig 3 deep in order to reach the string, 3 deep + 1 horizontal for the coordinates of said string

1/1 [==============================] - 1s 973ms/step


Old man's way of converting the recognized texts into sentences

In [51]:
text=''
text_list=[]
for i in range(0,test_batch):
    for j in range(0,len(pred_list[i])):
        text=text+' '+pred_list[i][j][0]
    text_list.append(text)
    text=''
text_list

[' when youre feeling horny habibi asf but is periods lets try goat your on a',
 ' club meanwhile the isis strip at']

The other way

In [42]:
import pandas as pd

In [59]:
df1=pd.DataFrame(pred_list[0],columns=('text','bbox'))
df1

,text,bbox
0,when,"[[84.0, 6.0], [161.0, 6.0], [161.0, 37.0], [84..."
1,youre,"[[161.30856, 7.490924], [249.76144, 4.889369],..."
2,feeling,"[[251.18549, 4.3947678], [351.5404, 6.842449],..."
3,horny,"[[354.35297, 6.411764], [436.7614, 9.158714], ..."
4,habibi,"[[161.0, 47.0], [233.0, 47.0], [233.0, 72.0], ..."
5,asf,"[[30.0, 48.0], [68.0, 48.0], [68.0, 73.0], [30..."
6,but,"[[68.0, 48.0], [108.0, 48.0], [108.0, 73.0], [..."
7,is,"[[232.0, 48.0], [257.0, 48.0], [257.0, 72.0], ..."
8,periods,"[[288.52917, 48.810467], [375.8194, 46.870678]..."
9,lets,"[[381.0, 48.0], [431.0, 48.0], [431.0, 73.0], ..."


In [60]:
sentence = ' '.join(df1['text'].tolist())
sentence

'when youre feeling horny habibi asf but is periods lets try goat your on a'

Will need to improve this later, happy this shit works and works WAYYYY BETTER

Ordering is a bit wack, but eh, it will work.

wooimbouatamakeawooimaboutahmakenameformyselfhere